In [1]:
from Bio.Blast import NCBIXML 
from Bio.Blast import NCBIWWW
import requests, sys, json
import re
from Bio import SeqIO
from Bio import Entrez
from Bio import Medline
import pandas as pd
import numpy as np
from tqdm import tqdm
from Bio import SearchIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from bs4 import BeautifulSoup

### web scraping (escolher condição e nº de genes)

In [12]:
query= input('escolha o que quer pesquisar: ')
def url_get(i):
    url_list= [ ]
    url = "https://www.ncbi.nlm.nih.gov/gene/?term={}".format( query )
    url_list.append(url)
    return url_list
url_get(1)

content = []
for url in url_get(1):
    r = requests.get(url)
    content.append(r.content)
#print(content)

# Parsing the HTML
for c in content:
    soup = BeautifulSoup(c, 'html.parser')
    a= soup.get_text()
#     print (a)

exist = re.findall(r"ID:\s+\d*(?=\D)", a, re.DOTALL)

c= ', '.join(exist)
h= c.replace('ID: ','')
IDS= h.split(', ')

n_genes= IDS[0:int(input('escolha o nº de genes que quer obter (máximo 20) atenção que o reultado pode ser menor que o desejado: '))] #can be more. related to ncbi pagination
print(n_genes)

if n_genes == ['']:
    print ("Não encontramos resultado para a sua pesquisa")
# else:
#     print (n_genes)

escolha o que quer pesquisar: irs1
escolha o nº de genes que quer obter (máximo 20) atenção que o reultado pode ser menor que o desejado: 10
['3667', '3667', '16367', '25467', '559281', '3077574', '100512686', '538598', '459985', '100607873']


### Buscar Links dos genes:

In [13]:
def url_get_id(i):
    url_list= [ ]
    for id in n_genes:
        url = "https://www.ncbi.nlm.nih.gov/nuccore/{}".format( id )
        url_list.append(url)
    return url_list
url_get_id(n_genes)

['https://www.ncbi.nlm.nih.gov/nuccore/3667',
 'https://www.ncbi.nlm.nih.gov/nuccore/3667',
 'https://www.ncbi.nlm.nih.gov/nuccore/16367',
 'https://www.ncbi.nlm.nih.gov/nuccore/25467',
 'https://www.ncbi.nlm.nih.gov/nuccore/559281',
 'https://www.ncbi.nlm.nih.gov/nuccore/3077574',
 'https://www.ncbi.nlm.nih.gov/nuccore/100512686',
 'https://www.ncbi.nlm.nih.gov/nuccore/538598',
 'https://www.ncbi.nlm.nih.gov/nuccore/459985',
 'https://www.ncbi.nlm.nih.gov/nuccore/100607873']

In [ ]:
    # FAZER HISTORICO?

### Buscar ids do ncbi:

In [14]:
Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    Ids.append(info.id)
print(len(Ids), Ids)

5 ['X03907.1', 'AA928418.1', 'AAADR0035516.1', 'L20655.1', 'AAADU0045050.1']


### Buscar quantidade de cds e a sua localização:

In [70]:
#original
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
featcds = [ ]
position=0
cds_location=[]
for info in records:
#     print(info.id)
    for i in range(len(info.features)):
        if info.features[i].type == "CDS":
            print(info.features[i].type == "CDS")
            featcds.append(i)
            cds=i
            a=str(cds)
#             print (len(a))
            position=i
            cds_location.append(info.features[position].location)
print(cds_location)
            
# print (pos)

True
True
[FeatureLocation(ExactPosition(145), ExactPosition(949), strand=1), FeatureLocation(BeforePosition(0), AfterPosition(92), strand=1)]


In [91]:
def get_CDS_location(id_list):
    database = 'nucleotide'
    email= 'rodrigoce9@gmail.com'
    Entrez.email = email
    handle = Entrez.efetch(db=database, id=id_list, rettype="gb") 
    records = list(SeqIO.parse(handle,"gb"))
    handle.close()
    cds_location_list = []
    for info in records:
        cds_location = []
        cds_location.append(info.id)
        for i in range(len(info.features)):
            if info.features[i].type == "CDS":
                cds_location.append(info.features[i].location)
        if len(cds_location)==1:
            cds_location.append(0)
        cds_location_list.append(cds_location)
    return cds_location_list
print(Ids)
# Ids.remove('U49845.1')
for i in get_CDS_location(Ids):
    print(i)

['X03907.1', 'AA928418.1', 'AAADR0035516.1', 'L20655.1', 'AAADU0045050.1', 'U49845.1']
['X03907.1', FeatureLocation(ExactPosition(145), ExactPosition(949), strand=1)]
['AA928418.1', 0]
['AAADR0035516.1', 0]
['L20655.1', FeatureLocation(BeforePosition(0), AfterPosition(92), strand=1)]
['AAADU0045050.1', 0]
['U49845.1', FeatureLocation(BeforePosition(0), ExactPosition(206), strand=1), FeatureLocation(ExactPosition(686), ExactPosition(3158), strand=1), FeatureLocation(ExactPosition(3299), ExactPosition(4037), strand=-1)]


In [94]:
Ids.append('NG_009030.2')

In [95]:
def get_CDS_location(id_list):
    database = 'nucleotide'
    email= 'rodrigoce9@gmail.com'
    Entrez.email = email
    handle = Entrez.efetch(db=database, id=id_list, rettype="gb") 
    records = list(SeqIO.parse(handle,"gb"))
    handle.close()
    cds_location_list = []
    for info in records:
        for i in range(len(info.features)):
            if info.features[i].type == "CDS":
                cds_location = []
                cds_location.append(info.id)
                cds_location.append(info.features[i].location)
                cds_location_list.append(cds_location)
    return cds_location_list
for i in get_CDS_location(Ids):
    print(i)

['X03907.1', FeatureLocation(ExactPosition(145), ExactPosition(949), strand=1)]
['L20655.1', FeatureLocation(BeforePosition(0), AfterPosition(92), strand=1)]
['U49845.1', FeatureLocation(BeforePosition(0), ExactPosition(206), strand=1)]
['U49845.1', FeatureLocation(ExactPosition(686), ExactPosition(3158), strand=1)]
['U49845.1', FeatureLocation(ExactPosition(3299), ExactPosition(4037), strand=-1)]
['NG_009030.2', CompoundLocation([FeatureLocation(ExactPosition(66454), ExactPosition(66602), strand=1), FeatureLocation(ExactPosition(196494), ExactPosition(196600), strand=1), FeatureLocation(ExactPosition(220376), ExactPosition(220579), strand=1), FeatureLocation(ExactPosition(224378), ExactPosition(224564), strand=1), FeatureLocation(ExactPosition(237847), ExactPosition(238049), strand=1), FeatureLocation(ExactPosition(239526), ExactPosition(239740), strand=1), FeatureLocation(ExactPosition(241074), ExactPosition(241197), strand=1), FeatureLocation(ExactPosition(244108), ExactPosition(244

In [122]:
from Bio.SeqFeature import CompoundLocation
def get_CDS_location(id_list):
    database = 'nucleotide'
    email= 'rodrigoce9@gmail.com'
    Entrez.email = email
    handle = Entrez.efetch(db=database, id=id_list, rettype="gb") 
    records = list(SeqIO.parse(handle,"gb"))
    handle.close()
    cds_location_list = []
    several_location = []
    for info in records:
        for i in range(len(info.features)):
            if info.features[i].type == "CDS":
                cds_location = []
                cds_location.append(info.id)
                if isinstance(info.features[i].location, CompoundLocation):
                    for sub_location in info.features[i].location.parts:
                        several_location.append("[{} : {}]".format(sub_location.start, sub_location.end))
                    cds_location.append(several_location)
                else:
                    cds_location.append("[{} : {}]".format(info.features[i].location.start, info.features[i].location.end))
                cds_location_list.append(cds_location)
    return cds_location_list
for i in get_CDS_location(Ids):
    print(i)

# < indicates 5' to 3' (forward direction)
# < indicates 3' to 5' (reverse direction)  one of the sugestion to improve work ++++++++++++++++++++++++++++++++

['X03907.1', '[145 : 949]']
['L20655.1', '[<0 : >92]']
['U49845.1', '[<0 : 206]']
['U49845.1', '[686 : 3158]']
['U49845.1', '[3299 : 4037]']
['NG_009030.2', ['[66454 : 66602]', '[196494 : 196600]', '[220376 : 220579]', '[224378 : 224564]', '[237847 : 238049]', '[239526 : 239740]', '[241074 : 241197]', '[244108 : 244243]', '[247567 : 247717]', '[251849 : 252014]', '[253604 : 253877]', '[255411 : 255748]', '[257341 : 259809]']]


### Buscar organismos (ainda em processo): 

In [25]:
Organismos=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    Ids.append(info.id)
    print(info.id)

X03907.1
AA928418.1
AAADR0035516.1
L20655.1
AAADU0045050.1


### Buscar Ids de proteinas

In [15]:
#original
list_pro=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    for i in info.features:
        if i.type == "CDS":
            pro= str(i.qualifiers["protein_id"])
            list_pro.append(pro)
div= ', '.join(list_pro)
h= div.replace("['",'')
hh= h.replace("']",'')
ID_PROT= hh.split(', ')
print (ID_PROT) # is used below in Uniprot

['CAA27540.1', 'AAA47844.1']


In [67]:
#FINAL
result_dict = {}
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist=Ids
# idlist= Ids.remove('U49845')
for ids in idlist:
    list_pro=[]
    handle = Entrez.efetch(db=database, id=ids, rettype="gb") 
    records = list(SeqIO.parse(handle,"gb"))
    handle.close()
    for info in records:
        list_pro.append(info.id)
        for i in info.features:
            if i.type == "CDS":
                pro= str(i.qualifiers["protein_id"])
                list_pro.append(pro)
        if len(list_pro)==1: #if no protein_id was found
            result_dict[info.id] = "N/A"
        else:
            list_pro = [x.replace("['",'').replace("']",'') for x in list_pro]
            result_dict[info.id] = ', '.join(list_pro[1:])
print(result_dict)
#key: id_genebank values: id_Uniprot

{'X03907.1': 'CAA27540.1', 'AA928418.1': 'N/A', 'AAADR0035516.1': 'N/A', 'L20655.1': 'AAA47844.1', 'AAADU0045050.1': 'N/A'}


In [27]:
#TESTE
lista =[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'

list_pro=[]
handle = Entrez.efetch(db=database, id='U49845', rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    list_pro.append(info.id)
    for i in info.features:
        if i.type == "CDS":
            pro= str(i.qualifiers["protein_id"])
            list_pro.append(pro)

div= ', '.join(list_pro)
h= div.replace("['",'')
hh= h.replace("']",'')
ID_PROT= hh.split(', ')
lista.append(ID_PROT)
print(lista) 

[['U49845.1', 'AAA98665.1', 'AAA98666.1', 'AAA98667.1']]


### Buscar Links de proteinas:

In [32]:
def url_get_id_p(dic):
    url_list_p=[ ]
    for key, value in dic.items():
        if
        url_id_p= "https://www.uniprot.org/uniprotkb?query={}".format(id_p)
        url_list_p.append(url_id_p)
    return url_list_p
url_ids_protein=url_get_id_p(result_dict)
print(url_ids_protein)  # but we don't need to save this links

['https://www.uniprot.org/uniprotkb?query=AAADU0045050.1']


In [63]:
def url_get_id_p(dic):
    url_list_p=[ ]
    for key, value in dic.items():
        if dic[key] !='N/A':
            url_id_p= "https://www.uniprot.org/uniprotkb?query={}".format(value)
            url_list_p.append(url_id_p)
    return url_list_p
url_ids_protein=url_get_id_p(result_dict)
print(url_ids_protein)  # but we don't need to save this links

['https://www.uniprot.org/uniprotkb?query=CAA27540.1', 'https://www.uniprot.org/uniprotkb?query=AAA47844.1']


### Buscar id proteina (uniprot):

In [65]:
def get_url(url, **kwargs):
    response = requests.get(url, **kwargs);

    if not response.ok:
        print(response.text)
        response.raise_for_status()
        sys.exit()

    return response

In [66]:
#final version
WEBSITE_API = "https://rest.uniprot.org"
fields = ["accession","organism_name","protein_name","cc_subcellular_location","cc_function", "sequence"]

def get_field_for_id(ID_PROT, field):
    response = get_url("{}/uniprotkb/search?query={} AND (reviewed:true)&fields={}&size=1&format=tsv".format(WEBSITE_API,ID_PROT,field))
    return str(response.content)


result = {}
for field in fields:
    result[field] = [get_field_for_id(i, field) for i in ID_PROT]

for field, values in result.items():
#     print(f"Results for {field}")
#     print(values)
    #regex
    for x in values:
        entry = re.search(r'b\'Entry\\n(.+?(?=\\n\'))', x, re.DOTALL)
        organism = re.search(r'b\'Organism\\n(.+?(?=\\n\'))', x, re.DOTALL)
        function = re.search( r'b\'Function \[(CC)\]\\n.{9} (.+?(?=\\n\'))', x, re.DOTALL )
        location = re.search( r'b\'Subcellular location .{27} (.+?(?=\\n\'))', x, re.DOTALL )
        sequence = re.search(r'b\'Sequence\\n(.+?(?=\\n\'))', x, re.DOTALL)
        if entry:
            e = re.match(r'b\'Entry\\n(.+?(?=\\n\'))', x, re.DOTALL )
            print(e.group(1))
        elif organism:
            o = re.match(r'b\'Organism\\n(.+?(?=\\n\'))', x, re.DOTALL )
            print(o.group(1))
        elif function:
            f = re.match(r'b\'Function .{15} (.+?(?=\\n\'))', x, re.DOTALL )
            print(f.group(1))
        elif location:
            l = re.match(r'b\'Subcellular location .{27} (.+?(?=\\n\'))', x, re.DOTALL )
            print(l.group(1))
        elif sequence:
            s = re.match(r'b\'Sequence\\n(.+?(?=\\n\'))', x, re.DOTALL)
            print(s.group(1) , len(s.group(1)) )
#         else:
#             print("fail")
#ADD CONDITION FOR EMPY RESULT (MEANS THE ID PROTEIN IS NOT REVIEWED)

### Buscar seq dos genes:

In [ ]:
#Biopython
Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    print(info.id)
    print(info.seq[0:50],'...',info.seq[-10:])
    print()
    seq.append(f'{info.seq[0:50]}...{info.seq[-10:]}')
    

### Buscar gene description:

In [ ]:
Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
description=[]
for info in records:
    print(info.id)
    print(info.description)
    description.append(info.description)

### Buscar artigos:

In [ ]:
Ids=[]
database = 'PubMed'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="medline", retmode="text") 
records = Medline.parse(handle)
description=[]
for info in records:
    print("title:", info.get("TI", "-"))
    print('authors: ', info.get("AU", "-"))
    print("source:", info.get("SO", "-"))
    print("affiliation: ", info.get("AD", "-") ) # FAZER O IF/ELSE PARA O CASO DE NÃO TER AFFILIATION
    print()